In [27]:
from threading import Thread

threads = list()
num_threads = 5

# create thread
function = lambda x: print(x ** 2)
for i in range(num_threads):
    # target is the function to run by the thread
    p = Thread(target=function(i))
    # add the thread to the list of threads
    threads.append(p)

# start the threads
# I am using list comprehension just for fun
[p.start() for p in threads]

# Each thread should wait others until they finish
# Here I am blocking the main thread until everything is done
[p.join() for p in threads]

# All of the above-mentioned threads will start in the same processes
# This is not a real parallelism, it's kinda thread switching

0
1
4
9
16


[None, None, None, None, None]

In [28]:
# Share data between threads
# Threads share the same memory space
import time
from threading import Lock

database_value = 0


def increase_db_value(lock):
    global database_value
    # acquire and release to create a critical sections
    lock.acquire()
    local_copy = database_value
    local_copy += 1
    time.sleep(0.1)
    database_value = local_copy
    lock.release()
    # or we can use with no need to release
    with lock:
        local_copy = database_value
        local_copy += 1
        time.sleep(0.1)
        database_value = local_copy


print(f"Start value {database_value}")
lock = Lock()
thread1 = Thread(target=increase_db_value, args=(lock,))
thread2 = Thread(target=increase_db_value, args=(lock,))

thread1.start()
thread2.start()

thread1.join()
thread2.join()
print(f"End value {database_value}")
# Notice that even if we are calling two threads to increase the value
# it's still 1
# Because we have a race condition
# Because we are reading the local copy before being updated
# We can use lock object

Start value 0
End value 4


In [31]:
# Queues are excellent for multithreading in python
# Queues are thread safe
from queue import Queue
from threading import currentThread

q = Queue()
num_threads = 5


def worker(q, lock):
    while True:
        # q.get() blocks until there is an available element
        value = q.  get()
        with lock:
            print(f'We are in {currentThread().name} and we got {value}')
            # To tell the program that you are done with a queue
        q.task_done()

lock = Lock()
for i in range(num_threads):
    t = Thread(target=worker, args=(q,lock))
    t.daemon = True
    t.start()

[q.put(i) for i in range(21)]
# To wait for processes in a queue to finish
q.join()
# A daemon thread dies when main thread dies

We are in Thread-113 and we got 0
We are in Thread-114 and we got 1
We are in Thread-115 and we got 2
We are in Thread-112 and we got 3
We are in Thread-112 and we got 8
We are in Thread-113 and we got 5
We are in Thread-114 and we got 6
We are in Thread-115 and we got 7
We are in Thread-115 and we got 12
We are in Thread-112 and we got 9
We are in Thread-113 and we got 10
We are in Thread-114 and we got 11
We are in Thread-114 and we got 16
We are in Thread-111 and we got 4
We are in Thread-111 and we got 18
We are in Thread-113 and we got 15
We are in Thread-114 and we got 17
We are in Thread-115 and we got 13
We are in Thread-111 and we got 19
We are in Thread-113 and we got 20
We are in Thread-112 and we got 14
